In [1]:
import sys
from sys.path.append('/Users/vvpitta/ece448/mp1/classes/')

from mclass import *
from maze import *
from minHeapClass import *

SyntaxError: invalid syntax (<ipython-input-1-dd1b0072804b>, line 2)

In [1]:
# /Users/vvpitta/ece448/mp1/inputMazes
maze, cellList, startIdx, goalIdx = getMaze('/openMaze.txt')
print maze + '\n'
print 'StartIdx =', startIdx
print 'GoalIdx =', goalIdx

NameError: name 'getMaze' is not defined

In [68]:
print 'Start X:', cellList[startIdx].x
print 'Start Y:', cellList[startIdx].y
print
print 'Goal X:', cellList[goalIdx].x
print 'Goal Y:', cellList[goalIdx].y

Start X: 23
Start Y: 1

Goal X: 9
Goal Y: 18


In [69]:
def ManhattanDistance(a, b):
    return abs(a.x - b.x) + abs(a.y - b.y)

def neighbors(maze, current):
    children = []
    if not maze[current.left].isWall:
        children.append(maze[current.left])
    if not maze[current.right].isWall:
        children.append(maze[current.right])
    if not maze[current.down].isWall:
        children.append(maze[current.down])
    if not maze[current.up].isWall:
        children.append(maze[current.up])
    return children

In [70]:
def A_Star_Search(maze, startIdx, goalIdx):
    openList = MinHeap()
    gVal = {}
    parents = {}
    count = 0
    
    openList.push(maze[startIdx], 0)
    gVal[maze[startIdx]] = 0
    parents[maze[startIdx]] = None
    
    while not openList.isEmpty():
        current = openList.pop()
        count += 1
        
        if current.idx == goalIdx:
            break
            
        children = neighbors(maze, current)
        for child in children:
            newGVal = gVal[current] + 1
            if child not in gVal or newGVal < gVal[child]:
                gVal[child] = newGVal
                parents[child] = current
                fn = newGVal + ManhattanDistance(child, maze[goalIdx])
                openList.push(child, fn)
    
    return gVal, parents, count

In [71]:
def drawPath(maze, cellList, parents, startIdx, goalIdx):
    retPath = []
    current = cellList[goalIdx]

    while current != cellList[startIdx]:
        retPath.append(current)
        current = parents[current]
    
    newMaze = list(maze)

    for movement in retPath:
        if movement.idx == startIdx:
            continue
        if movement.idx == goalIdx:
            continue
        newMaze[movement.idx] = '.'

    maze_string = ''.join(newMaze)
    print maze_string
    
    return

In [76]:
gVals, parents, expanded = A_Star_Search(cellList, startIdx, goalIdx)
drawPath(maze, cellList, parents, startIdx, goalIdx)
print gVals[cellList[goalIdx]]
print expanded

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P            %
%                     %.            %
%                     %.            %
%                     %.            %
%                     %........     %
%                     %%%%%%%%.     %
%                            %.     %
%                            %.     %
%                            %.     %
%                            %.     %
%     ...................    %.     %
%     .%%%%%%%%%%%%%%%%%.......     %
%     .%                            %
%     .%                            %
%     .%                            %
%     .%                            %
%     .%%%%                         %
%     ....%                         %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
53
371
